# Drake Systems Fundamentals

**Learning Objectives:**
1. Implement a custom `LeafSystem` from scratch
2. Understand Drake's block diagram systems framework
3. Build and wire block diagrams yourself
4. Write simulation code
5. Use Drake documentation and tutorials effectively

**What you'll implement:** You'll implement an inverted pendulum system from scratch, and simulate it using Drake's built-in simulation tools.

---


## Setup and Imports

Let us first import Drake functionality. We will cover them as we go along, so don't worry about them for now. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pydrake.all import (
    BasicVector,
    ConstantVectorSource,
    DiagramBuilder,
    LeafSystem,
    LogVectorOutput,
    Simulator,
)
from pydrake.systems.drawing import plot_system_graphviz

from manipulation.exercises.grader import Grader
from manipulation.exercises.intro.test_intro_fundamentals import (
    TestIntroFundamentalsPendulumImplementation,
    TestIntroFundamentalsSimulationExercises,
)

---

## Part 1: Writing Your Own Dynamics

Drake models complex systems as **block diagrams** composed of simple building blocks. Each block is a `System` that:
- Has **input ports** (receives signals)
- Has **output ports** (sends signals) 
- Maintains internal **state**
- Defines **dynamics** (how state evolves over time)

A `System` can be just a simple, single system, or it can be a collection of multiple smaller subsystems, for instance connected together in a `Diagram` (which we will learn more about in a second). For now, we will look only at a single custom system, which you will usually implement by inheriting from the Drake class `LeafSystem`. In this part of the notebook, you will implement your own simple physical system (an inverted pendulum) using `LeafSystem`. Later in the class and for the project you might find yourself inheriting from `LeafSystem` when you are writing custom controllers, estimators, sensors, etc!

**References:** To implement the tasks in this notebook, you will be referencing the official Drake tutorials. You don't have to read everything in them right now, we provide hints in the code pointing you towards the relevant sections. The tutorials we will be referencing are:
- [Dynamical Systems Tutorial](https://github.com/RobotLocomotion/drake/blob/master/tutorials/dynamical_systems.ipynb)
- [Authoring Leaf Systmes](https://github.com/RobotLocomotion/drake/blob/7abd7dc1a95387490e2d5fa23fe938f57eddecfc/tutorials/authoring_leaf_systems.ipynb)

If you are unsure about how to use a function, refer to the [Official Drake documentation](https://drake.mit.edu/pydrake/index.html). For example, the documentation for `DeclareVectorInputPort` and similar functions can be found [on this page](https://drake.mit.edu/pydrake/pydrake.systems.framework.html) (which we found by searching for "DeclareVectorInputPort) and then by cmd+F for "DeclareVectorInputPort".

**YOUR TASK:** Implement an inverted pendulum (with zero friction) as a custom `LeafSystem`
Here are the system specifications you will need:
- **State:** $x = [\theta, \dot{\theta}]$ (angle and angular velocity)
- **Input:** $u$ (torque applied at the base)  
- **Dynamics:** $\ddot{\theta} = -\frac{g}{l}\sin(\theta) + \frac{u}{ml^2}$
- **Output:** $y = \theta$ (just the angle)

In [ ]:
# TODO: Implement your own InvertedPendulum class


class InvertedPendulum(LeafSystem):
    def __init__(self, mass=1.0, length=1.0, gravity=9.81):
        # HINT: Look at the Dynamical Systems Tutorial, section "Deriving from LeafSystem"
        # TODO: Call the parent constructor

        # TODO: Store the physical parameters as instance variables

        # TODO: Declare continuous state for [theta, theta_dot]
        #       (note that we won't need the state_index they use in the tutorial,
        #        so you can disregard the return value)

        # HINT: Look at the Authoring Leaf Systems Tutorial, section "Vector-valued Ports"
        # TODO: Declare input port for torque
        # TODO: Save the input port as an instance variable so we can access it in `DoCalcTimeDerivatives`

        # TODO: Declare output port for **just** theta (not theta_dot)

        pass  # Remove this when you implement the TODOs

    def DoCalcTimeDerivatives(self, context, derivatives):
        # HINT: Look at the Dynamical Systems Tutorial, section "Deriving from LeafSystem"
        # TODO: Get current state from context (extract theta and theta_dot)
        # HINT: Note that context.get_continuous_state_vector() returns a VectorBase object,
        #       which you can index into to get the theta and theta_dot using .GetAtIndex(),
        #       or simply context.get_continuous_state_vector()[index]

        # HINT: Look at the Authoring Leaf Systems Tutorial, section "Vector-valued Ports"
        # TODO: Get input torque by evaluating the input port

        # HINT: Look at the Dynamical Systems Tutorial, section "Deriving from LeafSystem"
        # TODO: Compute pendulum dynamics

        # TODO: Set the derivatives [theta_dot, theta_ddot]

        pass  # Remove this when you implement the TODOs

    def OutputTheta(self, context, output):
        # HINT: Look at the Authoring Leaf Systems Tutorial, section "Vector-valued Ports"
        # TODO: Get state from context, and extract theta (first element))

        # TODO: Set the output port to theta

        pass  # Remove this when you implement the TODOs

**When you are done, make sure to run the tests below to make sure everything is working as expected!**

In [ ]:
Grader.grade_output(
    [TestIntroFundamentalsPendulumImplementation], [locals()], "results.json"
)
Grader.print_test_results("results.json")

### System `Context`
In the code above, you probably noticed the use of something called the `context`. In Drake, the `context` contains all the dynamical information about your simulation and all the systems in it.

Please read the following tutorial section before proceeding:
- [Dynamical Systems Tutorial](https://github.com/RobotLocomotion/drake/blob/master/tutorials/dynamical_systems.ipynb): **The System "Context"**

Next, let us create a default context for our Pendulum, change it a bit, and have a look at it!

For this simple system, you see that the context only contains the current time, as well as the two states we defined for our system.

In [ ]:
# TODO: Set the time and states for the context
# time =
# theta =
# theta_dot =

# TODO: Uncomment the following lines:
# pendulum = InvertedPendulum()
# pendulum_context = pendulum.CreateDefaultContext()
# pendulum_context.get_mutable_continuous_state().SetFromVector([theta, theta_dot])
# pendulum_context.SetTime(time)
# print(pendulum_context)

--- 
## Part 2: Connecting Systems in a Diagram

In this part, you will create a complete `Diagram` that we will later use for simulation by connecting multiple Drake `System`s together:
1. **InvertedPendulum** - your dynamical system
2. **ConstantVectorSource** - provides constant torque input  
3. **LogVectorOutput** - records output data over time

**YOUR TASK:** Build a `Diagram` for simulation of the inverted pendulum, where the pendulum is provided with a constant input torque, and we log the outputs.

**References:** In this part, we will only be referencing this tutorial (we will keep pointing you towards the relevant sections in the code):
- [Dynamical Systems Tutorial](https://github.com/RobotLocomotion/drake/blob/master/tutorials/dynamical_systems.ipynb): section **"Simulation"** and **"Combinations of Systems: Diagram and DiagramBuilder"**

In [ ]:
# TODO: Implement the diagram builder function


def build_pendulum_diagram(input_torque=0.0):
    """Build a complete block diagram for pendulum simulation."""

    # HINT: Look at the Dynamical Systems Tutorial, section "Simulation"
    # TODO: Create a DiagramBuilder

    # TODO: Add your inverted pendulum system to the builder

    # TODO: Add a constant torque source
    # HINT: This is not in the tutorial, so we have provided the code for you.

    # TODO: Add data logging system

    # TODO: Connect the systems together
    # HINT: Look at the Dynamical Systems Tutorial, section
    #       "Combinations of Systems: Diagram and DiagramBuilder"
    # HINT: The constant torque source should be connected to the pendulum's input port,
    #       but the LogVectorOutput is already connected to the pendulum's output port.

    # TODO: Build and return the final diagram, system, and logger

    pass  # Remove this when you implement


# TODO: Uncomment the following line:
# diagram, pendulum, logger = build_pendulum_diagram(input_torque=0.1)

**Next, we will visualize the block diagram.**

Drake can automatically generate graphical representations of your block diagrams, which can be very helpful for understanding the system connections and debug wiring issues.

Run the code below to visualize your diagram:

In [ ]:
# TODO: Uncomment the following line:
# diagram, pendulum, logger = build_pendulum_diagram(input_torque=0.1)
# plot_system_graphviz(diagram)

After visualizing, make sure to have a look at the diagram. Are the connections and the number of ports as you would expect?

Let us also have a look at the Diagram context (notice how it includes the pendulum context from earlier!):

In [ ]:
# TODO: Uncomment the following line:
# diagram, pendulum, logger = build_pendulum_diagram(input_torque=0.1)

# TODO: Create a context for the diagram

# TODO: Print the context for the diagram

**TIP:** This is not in the tutorial, but will be **very** useful soon in the class and for your project: 

If you ever want to get the context of a specific system from the entire diagram context, use the method:

`system.GetMyContextFromRoot(diagram_context)`.

Try it below for the inverted pendulum, and notice how we get back the simple pendulum context we printed earlier!


In [ ]:
# TODO: Get the context of the pendulum from the diagram context

# TODO: Print the pendulum context

## Part 3: Simulation
In this part, we will finally put everything together and run the full simulation of our pendulum.

**YOUR TASK:** Implement the full simulation for your inverted pendulum system.

**References:** In this part, we will only be referencing this tutorial (we will keep pointing you towards the relevant sections in the code):
- [Dynamical Systems Tutorial](https://github.com/RobotLocomotion/drake/blob/master/tutorials/dynamical_systems.ipynb): section **"Simulation"**

In [ ]:
# TODO: Implement the simulation function


def simulate_pendulum(initial_state, simulation_time=5.0, torque=0.0):
    """Simulate the pendulum with given initial conditions."""

    # TODO: Build your diagram using the function you implemented

    # HINT: Look at the Dynamical Systems Tutorial, section "Simulation"
    # TODO: Create a context for the diagram

    # TODO: Create a simulator for the diagram with the context

    # TODO: Set initial conditions for the pendulum

    # TODO: Run the simulation to the specified time

    # TODO: Extract the logged data from the logger

    # TODO: Return the time and state data from the log

    pass  # Remove this when you implement

When you are done, let us run the code below to test the implementation. We will check that the pendulum swings past the downright equilibrium point, and also do some visualization so you can see what is happening!

In [ ]:
# Test your implementation (you do not need to modify this code)
initial_state = [0.5, 0.0]  # small angle, zero velocity
times, outputs = simulate_pendulum(initial_state, simulation_time=10.0, torque=0.0)

print(f"Simulation complete! Recorded {len(times)} timesteps")
print(f"Initial angle: {outputs[0, 0]:.4f} rad ({np.degrees(outputs[0, 0]):.1f}°)")
print(f"Final angle: {outputs[0, -1]:.4f} rad ({np.degrees(outputs[0, -1]):.1f}°)")

# Check if pendulum swung past the equilibrium point
min_angle = np.min(np.abs(outputs[0, :]))
if np.isclose(min_angle, 0.0, atol=1e-3):
    print("✅ Physics check PASSED - pendulum oscillated as expected")
else:
    print("❌ Physics check FAILED - pendulum didn't swing properly")

print(
    "Let us plot the angle over time (notice how there is no friction, hence the pendulum swings indefinitely!):"
)
# Plot the angle over time
plt.figure(figsize=(10, 4))
plt.plot(times, outputs[0, :], "b-", linewidth=2, label="θ (angle)")
plt.xlabel("Time (s)")
plt.ylabel("Angle (rad)")
plt.title("Inverted Pendulum Angle vs Time")
plt.grid(True, alpha=0.3)
plt.legend()

# Add degree labels on right y-axis
ax2 = plt.gca().twinx()
ax2.set_ylabel("Angle (degrees)")
ax2.set_ylim(np.degrees(plt.gca().get_ylim()[0]), np.degrees(plt.gca().get_ylim()[1]))

plt.tight_layout()
plt.show()

---

# VERIFICATION IN GRADESCOPE 

**Prerequisites:** You must complete ALL the TODOs above before these verification exercises will work!

**Instructions:** Implement the exercises below. Copy the exact numerical values (to 4 decimal places) for your verification keys, which you can copy/paste to Gradescope.



## Verification 1: Basic Simulation

**Task:** Simulate the inverted pendulum with:
- Initial state: θ = 0.15 rad, θ̇ = 0.0 rad/s  
- No applied torque (torque = 0.0)
- Simulation time: 2.5 seconds

**Question:** What is the angle θ at t = 2.5 seconds? (Report to 4 decimal places)


In [ ]:
# TODO: Implement the simulation exercise

## Verification 2: With Applied Torque

**Task:** Simulate the pendulum with:
- Initial state: θ = -0.1 rad, θ̇ = 0.2 rad/s
- Constant applied torque: 0.5 N⋅m
- Simulation time: 1.8 seconds

**Question:** What is the angular velocity θ̇ at t = 1.8 seconds? (Report to 4 decimal places)


In [ ]:
# TODO: Implement the simulation exercise

---

# Congratulations!

You've successfully completed **Drake Systems Fundamentals**! You've learned:

1. **LeafSystem** is the base class for most custom Drake systems
3. **Context** holds the state, time, and parameters for simulation
2. **DiagramBuilder** connects systems into larger compositions (`Diagram`s)
4. **Drake documentation** and **Drake tutorials** are your best friends - use them!

**Next:** In Notebook 2, you'll work with real robots using `MultibodyPlant` and create your own custom assets!


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2790fa1a-af91-49f3-bf0b-84addd6622d9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>